In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import xsimlab as xs

# to measure process time
import time as tm

In [2]:
import attr

In [3]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

from phydra.processes.main import ThirdInit

from phydra.processes.statevars import SV, Time

from phydra.processes.forcings import ConstantForcing

from phydra.processes.fluxes import LossLinearFlux, ForcingInputLinearFlux

from phydra.core.parts import Parameter

In [6]:
# what should the final wrapped class interface look like? 
# in what way 

# SO THE FIRST (and maybe only) way to implemented is for Fluxes!
# the basic types of fluxes have their decorators
# and each flux uses one of these decorators and
# the actual function is a simple flux function

# so how can I turn a simple (latexifiable) function, into an xs.process via decoration?

hm... so now I have this simple wrapper, but it don't make any sense..

# also no latexification as of right now, unless modifying base code

what can I do to make this more senseful?

# no. no. 
good idea: instead of mix matching base class & flux func
    
instead create abc for flux, that includes inputs outputs and parameters

and from this the proper flux PROCESS is created!

# latexify_py can be incorporated into this class! compatibility with base math

xs.variable returns attr.attrib("content here")

# 

SyntaxError: invalid syntax (<ipython-input-6-778419129297>, line 11)

In [4]:
from phydra.processes.main import ThirdInit

In [5]:
from collections import OrderedDict

In [6]:
from enum import Enum

class FluxVarType(Enum):
    STATEVARIABLE = "statevariable"
    PARAMETER = "parameter"
    FORCING = "forcing"

class FluxVarIntent(Enum):
    INPUT = "input"
    OUTPUT = "output"

In [29]:
def _make_property_flux(flux):
    return xs.variable(intent='in')


class _FluxProcessBuilder:
    """ 
    Process builder creates xs.process, from Flux base class
    """
    
    _make_prop_funcs = {
        FluxVarType.STATEVARIABLE: _make_property_flux,
        FluxVarType.PARAMETER: _make_property_flux,
        FluxVarType.FORCING: _make_property_flux,
    }
    
    def __init__(self, cls):
        print("INIT:", type(cls), cls)
        self._base_cls = ThirdInit  # handles init structure (fluxes initialized last)
        self._flux_cls = cls
        self._flux_func = cls.flux
        self._flux_cls_dict = {}
        
        # maybe here, add a dict that builds the flux function later on
        # maybe that stores input output, and parameters, forcings
        # flux cls dict stores xs.variables and setup!
    
    
    def _reset_attributes(self):
        new_attributes = OrderedDict()
        print("resetting attrs")
        
        for k, attrib in attr.fields_dict(self._flux_cls).items():
            print(k, attrib)
            new_attributes[k] = attr.attrib(
                metadata=attrib.metadata,
                validator=attrib.validator,
                converter=attrib.converter,
                default=attrib.default,
                init=False,
                repr=False,
            )
        
        print("reset attrs", new_attributes)
        return new_attributes
    
    
    def add_properties(self):
        for var_name, var in attr.fields_dict(self._flux_cls).items():
            var_type = var.metadata.get("var_type")
            
            print("add_properties", var_type)
            
            if var_type is not None:
                make_prop_func = self._make_prop_funcs[var_type]
                print(make_prop_func(var))
                self._flux_cls_dict[var_name] = make_prop_func(var)
                
    
    def _make_xsimlab_process(self):
        print("MAKING xsimlab process")
        p_cls = attr.make_class(
            self._flux_cls.__name__,
            self._reset_attributes(),
            bases=(self._base_cls,),
            init=False,
            repr=False,
        )
        print("P_CLSS", p_cls)
        return p_cls
    
    def _setup_flux_func(self):
        
        print(self._flux_func)
        
        return self._flux_func
    
    
    def _setup_init_func(cls):
        
        def initialize(self):
            super(getattr(self, '__class__'), self).initialize()  # handles initialization stages
            
            print(f"initializing flux {self.label}")
            print("INIT parent wrapped class: ", cls)
            print("INIT self: ", self)
            
            print(getattr(self, '__class__'))
            
            print(getattr(self, '__xsimlab_name__'))
            
            # setup parameter
            self.m.Parameters[self.label + '_rate'] = Parameter(name=self.label + '_rate', value=self.rate)
            # create flux
            self.m.Fluxes[self.sv_label].append(self.flux)
            
        return initialize
    
    
    def create_process(self):
        
        p_cls = self._make_xsimlab_process()
        
        for name, value in self._flux_cls_dict.items():
            print("from flux cls dict", name, value)
            setattr(p_cls, name, value)
        
        setattr(p_cls, "initialize", self._setup_init_func())
        setattr(p_cls, "flux", self._setup_flux_func())
        
        return xs.process(p_cls)


In [30]:
def processfromflux(maybe_cls=None):
    
    def wrap(cls):
        print("wrapper cls", cls)
        
        attr_cls = attr.attrs(cls, repr=False)
        
        print("wrapper attr cls", attr_cls)
        
        builder = _FluxProcessBuilder(attr_cls)

        builder.add_properties()

        #if autodoc:
        #    builder.render_docstrings()

        return builder.create_process()
    
    
    if maybe_cls is None:
        return wrap
    else:
        return wrap(maybe_cls)

In [31]:
@processfromflux
class XXXX:
    sv_label = attr.ib(metadata={"var_type": FluxVarType.STATEVARIABLE})
    
    rate = attr.ib(metadata={"var_type": FluxVarType.PARAMETER})
    
    def flux(self):
        print("FLUXXING")
        print(self.sv, self.rate)
        
        
        pass
    
    #def flux(self, **kwargs):
    #    state = kwargs.get('state')
    #    parameters = kwargs.get('parameters')
    #
    #    sv = state[self.sv_label]
    #    rate = parameters[self.label + '_rate']
    #    delta = -rate * sv
    #    return delta

wrapper cls <class '__main__.XXXX'>
wrapper attr cls <class '__main__.XXXX'>
INIT: <class 'type'> <class '__main__.XXXX'>
add_properties FluxVarType.STATEVARIABLE
_CountingAttr(counter=143, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})
add_properties FluxVarType.PARAMETER
_CountingAttr(counter=145, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})
MAKING xsimlab process
resetting attrs
sv_label Attribute(name='sv_label', default=NOTHING, validator=None, repr=True, eq=True, order=True, hash=None, init=True, metadata=mappingproxy({'var_type': <FluxVarType.STATEVARIABLE: 'statevariable'

In [32]:
xs.process_info(XXXX)

<XXXX  (xsimlab process)>
Variables:
    m              [in] <--- ModelCore.m
    label         [out]
    firstinit      [in] <--- group 'FirstInit'
    secondinit     [in] <--- group 'SecondInit'
    group         [out]
    sv_label       [in]
    rate           [in]
Simulation stages:
    initialize


In [33]:
Model = phydra.create({
    # SV
    'y':SV,
    'x':SV, 
    
    # Fluxes
    'y_loss':XXXX,
    'x_loss':XXXX,
    
    # Forcing
    'y_input':ForcingInputLinearFlux,
    'N0':ConstantForcing
})

In [34]:
Model

<xsimlab.Model (9 processes, 12 inputs)>
Core
    solver_type     [in]
Time
    time            [in] ('input_time',) A sequence of Time points ...
y
    init            [in]
x
    init            [in]
N0
    value           [in]
y_loss
    rate            [in]
    sv_label        [in]
x_loss
    rate            [in]
    sv_label        [in]
y_input
    rate            [in]
    fx_label        [in]
    sv_label        [in]
Solver

In [35]:
#Model.visualize(show_inputs=True)

In [36]:
odeint_setup = phydra.setup('odeint', Model,
            time=np.arange(0,20,0.1),
            input_vars={
                'y__init':0.5,
                'x__init':0.5,
                
                'y_loss__sv_label':'y',
                'y_loss__rate':0.01,
                
                'x_loss__sv_label':'x',
                'x_loss__rate':0.5,
                
                'y_input':{'sv_label':'y',
                          'fx_label':'N0',
                          'rate':.1},
                'N0__value':2.
            },
            output_vars={
                'y__value':None,
                'x__value':None,
            })

In [37]:
out_ds = odeint_setup.xsimlab.run(model=Model)

initializing model core
Initializing Model Time
initializing state variable y
initializing state variable x
initializing forcing N0


RecursionError: maximum recursion depth exceeded while calling a Python object

In [28]:
out_ds.y__value.plot()
out_ds.x__value.plot()

NameError: name 'out_ds' is not defined

In [110]:
@processfromflux
class fluxxx:
    pass

fluxxx

__main__.LinearFlux

In [44]:
@xs.process
class LinearFlux(ThirdInit):
    """represents a flux in the model"""

    sv_label = xs.variable(intent='in')
    rate = xs.variable(intent='in')

    def initialize(self):
        super(LinearFlux, self).initialize()  # handles initialization stages
        print(f"initializing flux {self.label}")

        # setup parameter
        self.m.Parameters[self.label + '_rate'] = Parameter(name=self.label + '_rate', value=self.rate)
        # create flux
        self.m.Fluxes[self.sv_label].append(self.flux)

NameError: name 'ThirdInit' is not defined

In [34]:
#@phydra.flux
def flux():
    pass

In [36]:
flux()

In [89]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [3]:
import phydra.utility.latexify as latexify

In [29]:
def hello(a,b,c):
    return a + c/b + c

In [33]:
latexify.with_latex(hello)

<phydra.utility.latexify.core.with_latex.<locals>._LatexifiedFunction at 0x15fa594f0>

In [39]:
@process(*args)
class HELLO:
    
    Hey = xs.variable()

NameError: name 'args' is not defined

In [35]:
xs.process_info(HELLO)

<HELLO  (xsimlab process)>
Variables:
    Hey         [in]
Simulation stages:
    *no stage implemented*


In [ ]:
def make_flux(flux_cls, flux_name):
    """
    This functions creates a properly labeled xs.process from class Component.
    :args:
        cls_name (cls): Class of forcing flux to be initialized
        dim_name (str): Name of sub-dimension of returned process, UPPERCASE!
    :returns:
        xs.process of class Component
    """
    new_dim = xs.index(dims=(flux_name), groups='flux_index')
    base_dict = dict(flux_cls.__dict__)
    base_dict[flux_name] = new_dim

    new_cls_name = flux_cls.__name__ + '_' + flux_name
    new_cls = type(new_cls_name, flux_cls.__bases__, base_dict)

    
    def initialize(self):
        super(LinearFlux, self).initialize()  # handles initialization stages
        print(f"initializing flux {self.label}")

        # setup parameter
        self.m.Parameters[self.label + '_rate'] = Parameter(name=self.label + '_rate', value=self.rate)
        # create flux
        self.m.Fluxes[self.sv_label].append(self.flux)
        
        
    def initialize_dim(self):
        r_label = getattr(self, 'R_label')
        c_label = getattr(self, 'C_label')

        cls_label = getattr(self, '__xsimlab_name__')
        setattr(self, 'cls_label', cls_label)
        print(f"flux {cls_label} is initialized for {r_label} --> {c_label}")

        setattr(self, 'flux_label', str(cls_label))
        fx_c_list = []
        for r_lab, c_lab in zip(r_label, c_label):
            if self.gk_SVshapes[r_lab].size == 1 and self.gk_SVshapes[c_lab].size == 1:
                fx_c_list.append(f"{cls_label}-{r_lab}-2-{c_lab}")
            elif self.gk_SVshapes[r_lab].size == 1:
                for c in range(self.gk_SVshapes[c_lab].size):
                    fx_c_list.append(f"{cls_label}-{r_lab}-2-{c_lab}{c}")
            else:
                for r in range(self.gk_SVshapes[r_lab].size):
                    for c in range(self.gk_SVshapes[c_lab].size):
                        fx_c_list.append(f"{cls_label}-{r_lab}{r}-2-{c_lab}{c}")

        setattr(self, flux_name, fx_c_list)

        cls_here = getattr(self, '__class__')
        super(cls_here, self).initialize_parametersetup()

        
    setattr(new_cls, 'initialize', initialize_dim)

    
    if flux_name.lower() == flux_name:
        raise ValueError(f"dimension label ({flux_name}) supplied to forcing flux {flux_cls} needs to be Upper Case")

        
    new_cls.output.metadata['dims'] = ((flux_name, 'time'),)
    return xs.process(new_cls)

In [ ]:
def make_flux(flux_cls, flux_name):
    """
    This functions creates a properly labeled xs.process from class Component.
    :args:
        cls_name (cls): Class of forcing flux to be initialized
        dim_name (str): Name of sub-dimension of returned process, UPPERCASE!
    :returns:
        xs.process of class Component
    """
    new_dim = xs.index(dims=(flux_name), groups='flux_index')
    base_dict = dict(flux_cls.__dict__)
    base_dict[flux_name] = new_dim

    new_cls_name = flux_cls.__name__ + '_' + flux_name
    new_cls = type(new_cls_name, flux_cls.__bases__, base_dict)

    def initialize_dim(self):
        r_label = getattr(self, 'R_label')
        c_label = getattr(self, 'C_label')

        cls_label = getattr(self, '__xsimlab_name__')
        setattr(self, 'cls_label', cls_label)
        print(f"flux {cls_label} is initialized for {r_label} --> {c_label}")

        setattr(self, 'flux_label', str(cls_label))
        fx_c_list = []
        for r_lab, c_lab in zip(r_label, c_label):
            if self.gk_SVshapes[r_lab].size == 1 and self.gk_SVshapes[c_lab].size == 1:
                fx_c_list.append(f"{cls_label}-{r_lab}-2-{c_lab}")
            elif self.gk_SVshapes[r_lab].size == 1:
                for c in range(self.gk_SVshapes[c_lab].size):
                    fx_c_list.append(f"{cls_label}-{r_lab}-2-{c_lab}{c}")
            else:
                for r in range(self.gk_SVshapes[r_lab].size):
                    for c in range(self.gk_SVshapes[c_lab].size):
                        fx_c_list.append(f"{cls_label}-{r_lab}{r}-2-{c_lab}{c}")

        setattr(self, flux_name, fx_c_list)

        cls_here = getattr(self, '__class__')
        super(cls_here, self).initialize_parametersetup()

    setattr(new_cls, 'initialize', initialize_dim)

    if flux_name.lower() == flux_name:
        raise ValueError(f"dimension label ({flux_name}) supplied to forcing flux {flux_cls} needs to be Upper Case")

    new_cls.output.metadata['dims'] = ((flux_name, 'time'),)
    return xs.process(new_cls)